# The Knapsack Problem 
In the knapsack problem (KP), a set of items with associated weights and values should be stored in a knapsack. The problem is to maximize the value of the items transported in the knapsack. The KP is restricted by the maximum weight the knapsack can carry. The KP is the simplest nontrivial integer programming model with binary variables, only one constraint, and positive coefficients. It is formally defined by

\begin{equation}\label{Eq.1}
\max  \sum_{i=1}^{n} p_{i} x_{i},
\end{equation}

\begin{equation}\label{Eq.2}
\sum_{i=1}^{n} w_{i} x_{i} \leq W, 
\end{equation}

where $n$ is the number of items, $p_{i}$ and $w_{i}$ is the value and weight of the $ith$ item, respectively, $x_i$ is the binary variable that represents whether the $ith$ item is in the knapsack or not, and W is the maximum weight that the knapsack can transport. 

In this tutorial, we will generate an instance of the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem) and solve it using the quantum approximate optimization algorithm [QAOA](https://arxiv.org/abs/1411.4028). Our goal is to undestand the different steps to encode a combinatorial optimization problem as an ising Hamiltonian, how the QAOA function works and how postprocessing the results of QAOA.

# Table of Contents

1. [Quadratic unconstrained binary optimization](#qubo)
2. [Example](#example)<br>
    2.1 [Ising Hamiltonian](#ising)<br>
    2.2 [Brute force solution](#brute_sol)
3. [QAOA](#qaoa)<br>
    3.1 [QAOA circuit](#qaoa_circ)<br>
    3.2 [Optimization](#qaoa_opt)<br>
    3.3 [Visualization](#visualization)
4. [Task 3](#task3)

## <a class="anchor" id="qubo"> </a>1. Quadratic unconstrained binary optimization (QUBO)

The set of combinatorial problems that can be represented by the QUBO formulation are characterized by functions of the form

\begin{equation}
f(\mathrm{x}) = \frac{1}{2}\sum_{i=1}^{n} \sum_{j=1}^n q_{ij} x_{i} x_{j}, \tag{1}
\end{equation}
where $n$ is the number of variables, $q_{ij} \in \mathbb{R}$ are coefficients associated to the specific problem, and $x_i \in \{0,1\}$ are the binary variables of the problem. Note that $x_{i} x_{i} \equiv x_{i}$ and $q_{ij} = q_{ji}$ in this formulation. Therefore, the general form of a combinatorial optimization problem solvable by QPUs is given by the cost function

\begin{equation}\label{QUBO_form}
f(\mathrm{x}) = \sum_{i=1}^{n-1} \sum_{j > i}^n q_{ij}x_{i}x_{j} + \sum_{i=1}^n q_{ii} x_i,\tag{2}
\end{equation}
and equality constraints given by

\begin{equation}
\sum_{i=1}^n c_i x_i = C, \ c_i \in \mathbb{R}, \tag{3}
\end{equation}
and inequality constraints given by

\begin{equation}\label{inequality}
\sum_{i=1}^n l_i x_i \ge B, \ l_i \in \mathbb{R} \tag{4}
\end{equation}

where $C$ and $R$ are constants. To transform these problems into the QUBO formulation the constraints are added as penalization terms. In this respect, the equality constraints are included in the cost function using the following penalization term

\begin{equation}\label{EQ_F}
\lambda_0 \left(\sum_{i=1}^n c_i x_i - C\right)^2,\tag{5}
\end{equation}
where $\lambda_0$ is a penalization coefficient that should be chosen to guarantee that the equality constraint is fulfilled. In the case of inequality constraint, the common approach is to use a [slack variable](https://en.wikipedia.org/wiki/Slack_variable#:~:text=In%20an%20optimization%20problem%2C%20a,constraint%20on%20the%20slack%20variable.). The slack variable, $S$, is an auxiliary variable that makes a penalization term vanish when the inequality constraint is achieved,

 \begin{equation}\label{ineq}
 \sum_{i=1}^n l_i x_i - S - B = 0.\tag{6}
 \end{equation}
 Therefore, when Eq.(\ref{inequality}) is satisfied, Eq.(\ref{ineq}) is already zero. This means the slack variable, $S$, must be in the range $0 \le S \le \max_x \sum_{i=1}^n l_i x_i - B$. To represent the $slack$ variable in binary form, the slack is decomposed in 
 binary variables: 

\begin{equation}\label{SB} 
S = \sum_{k=0}^{N-1} 2^k s_k,\tag{7}
\end{equation}
where $s_k$ are the slack binary variables. Then, the inequality constraints are added as penalization terms by 
 
 \begin{equation}\label{Ineq_EF}
 \lambda_1  \left(\sum_{i=1}^n l_i x_i - \sum_{k=0}^{N-1} 2^k s_k - B\right)^2. \tag{8}
 \end{equation}
 
Combining Eq.(\ref{QUBO_form}) and the two kinds of constraints Eq.(\ref{EQ_F}) and Eq.(\ref{Ineq_EF}), the general QUBO representation of a given combinatorial optimization problem is given by

 \begin{equation}\label{QUBO}
 \min_x \left(\sum_{i=1}^{n-1} \sum_{j > i}^nc_{ij}x_{i}x_{j} + \sum_{i=1}^n h_i x_i + \lambda_0  \left(\sum_{i=1}^n q_i x_i - C\right)^2
+  \lambda_1  \left(\sum_{i=1}^n l_i x_i - \sum_{k=0}^{N-1} 2^k s_k - B\right)^2\right). \tag{10}
 \end{equation}

Following the same principle, more constraints can be added and note that after some manipulations, Eq.(\ref{QUBO}) can be rewritten in the form of Eq.(\ref{QUBO_form}). The last step to represent the QUBO problem on QPUs is to change the $x_i$ variables to spin variables $z_i \in \{1, -1\}$ by the transformation $x_i = (1 - z_i) / 2$. Hence, Eq.(\ref{QUBO_form}) represented in terms of the cost Hamiltonian model reads

\begin{equation}\label{IsingH}
H_c(\mathrm{z}) = \sum_{i=1}^{n-1}\sum_{j>i}^n q_{ij} (1 - z_i) (1 - z_j)/4 + \sum_{i=1}^n q_{ii} (1 - z_i)/2.\tag{11}
\end{equation} 

In [1]:
from sympy import Symbol
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit import Aer
from qiskit.visualization import plot_histogram

label_size = 16
plt.rcParams['xtick.labelsize'] = label_size 
plt.rcParams['ytick.labelsize'] = label_size 
plt.rcParams['axes.labelsize'] = label_size 
plt.rcParams['legend.fontsize'] = label_size 
%matplotlib inline

### <a class="anchor" id="example"> </a>2. Example

A knapsack problem with 3 items with weights $w_i = [1, 2, 3]$, values $v_i=[5, 2, 4]$, and the knapsack maximum weight $W_{max}=3$, 

\begin{equation}\label{QUBO_form}
f(\mathrm{x}) = \sum_{i=1}^{3} v_{i}x_{i} \tag{12}
\end{equation}

and inequality constraints given by

\begin{equation}\label{inequality}
W_{max} - \sum_{i=1}^3 w_i x_i \ge 0, \tag{14}
\end{equation}

The problem has a QUBO formulation given by Eq. \ref{QUBO_K}
\begin{equation}\label{QUBO_K}
\min_x \sum_{i=1}^n -v_i x_i + \lambda_1  \left( W_{max} - \sum_{i=1}^{3}w_i x_i -\sum_{k=0}^{N-1} 2^k s_k \right)^2, \tag{15}
\end{equation}

where $N = \lceil \log_2(\max_x W_{max} - \sum_{i=1}^n w_i x_i)\rceil = \log_2(W_{max})$.

In [ ]:
def Knapsack(values: list, weights: list, max_weight: int, penalty:float):
    n_items = len(values) # number of variables
    n_slacks = int(np.ceil(np.log2(max_weight))) # number of slack variables
    
    x = {i: Symbol(f"x{i}") for i in range(n_items)}  # variables that represent the items
    S = sum(2**k * Symbol(f"s{k}") for k in range(n_slacks)) # the slack variable in binary representation
    
    # objective function --------
    cost_fun = - sum([values[i]*x[i] for i in x]) # maximize the value of the items trasported Eq.12
    #(Note that minimizing the negative of cost function is the same that maximizing it)
    
    # ---------    constraint   Eq. 14  ----------
    constraint = max_weight - sum(weights[i] * x[i] for i in x) - S #inequality constraint

    cost = cost_fun + penalty * constraint**2 # Eq. 15 cost function with penalization term for the Knapsack problem
    return cost

In [ ]:
values = [5, 2, 4]
weights = [1, 2, 3]
max_weight = 3
penalty = 5 #lambda_1
qubo = Knapsack(values, weights, max_weight, penalty) # Eq. 10 QUBO formulation
print(r'QUBO: min_x', qubo)

Note that $x_{i} x_{i} \equiv x_{i}$, therefore

In [ ]:
# Expanding and replacing the quadratic terms xi*xi = xi
qubo = qubo.expand().subs({symbol**2:symbol for symbol in qubo.free_symbols})
qubo

### <a class="anchor" id="ising"> </a> 2.1 Ising Hamiltonian
The last step to represent the QUBO problem on QPUs is to change the $x_i \in \{0, 1\}$ variables to spin variables $z_i \in \{-1, 1\}$ by the transformation 

$x_i = (1 + z_i) / 2$    (Eq. 11)


### <font color=#C4232E> Exercise 1 </font>

Using the [`subs`](https://www.geeksforgeeks.org/python-sympy-subs-method-2/) sympy function, convert the QUBO formulation into an `Ising Hamiltonian`. You can use the `new_vars` dictionary provided below.

In [ ]:
new_vars = {xi:(1 + Symbol(f"z{i}"))/2 for i, xi in enumerate(qubo.free_symbols)}
new_vars

In [ ]:
ising_Hamiltonian = # your code here
print("H(z)=", ising_Hamiltonian)

### <a class="anchor" id="brute_sol"> </a> 2.2 Brute force solution

The first option to solve the knapsack problem is to use the brute force method. This method evaluates all the possible solutions of the QUBO and returns the one with the minimum cost. However, the number of possible solutions scales as $2^n$ where $n$ is the number of items in the problem, this makes this solution unfeasible for large instances.

### <font color=#C4232E> Exercise 2 </font>

Evaluate the `QUBO` for all possible bitstrings and return the bitstring with the lowest energy and the lowest energy. 

In [ ]:
def brute_force(qubo):

    #your code here
    
    return min_cost, bitstring

sol_brute = brute_force(qubo)
optimal = {var:int(s) for var, s in zip(qubo.free_symbols, sol_brute[1][1])}
sol_str = sol_brute[1][1]
print(f"Optimal result: {optimal} | cost:{sol_brute[1][0]}")

## <a class="anchor" id="qaoa"> </a>3. QAOA Circuit

Finally, we use [QAOA](https://arxiv.org/pdf/1411.4028.pdf) to find the solution to our Knapsack problem. In this case, the cost Hamiltonian, $H(z)$, obtained from the QUBO formulation, is translated into a parametric unitary gate given by

\begin{equation}\label{UC}
    U(H_c, \gamma)=e^{-i \gamma H_c},\tag{16}
\end{equation}
 where $\gamma$ is a parameter to be optimized. A second unitary operator applied is 

\begin{equation}\label{UB}
    U(B, \beta)=e^{i \beta X},\tag{17}
\end{equation}

where $\beta$ is the second parameter that must be optimized and $X = \sum_{i=1}^n \sigma_i^x$ with $\sigma_i^x$ the Pauli\-x quantum gate applied to qubit $i$. The general QAOA circuit is shown in **Fig.1**. Here, $R_X(\theta) = e^{-i \frac{\theta}{2} \sigma_x}$, $p$ represents the number of repetitions of the unitary gates Eqs.\ref{UC} and \ref{UB} with each repetition having separate values for $\gamma_p$ and $\beta_p$, and the initial state is a superposition state $| + \rangle^{\otimes n}$.

<br><center><img src="./Images/QAOA.png" width="400"><img src="./Images/Circuit_Requirements.png" width="300"><b> **Fig.1** Schematic representation of QAOA for $p$ layers. The parameters $\gamma$ and $\beta$ for each layer are the ones to be optimized. </center><br>


### <font color=#C4232E> Exercise 3 </font>

Complete the code in the `QAOA function`. You can use [`rz`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZGate.html) gate and [`rzz`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html) gate.

In [ ]:
def qaoa_circuit(ising_Hamiltonian, reps=1):
    gammas = ParameterVector(r"$\gamma$", reps) # Create a set of variables gamma
    betas = ParameterVector(r"$\beta$", reps) # Create a set of variables beta
    terms_and_weights = sympy_to_dict(ising_Hamiltonian) #convert the hamiltonian into a dictionary
    num_qubits = len(ising_Hamiltonian.free_symbols) # Based on the variables 
    qc = QuantumCircuit(num_qubits) # Creating a quantum circuit
    # Apply the initial layer of Hadamard gates to all qubits
    qc.h(range(num_qubits))
    # repeat p layers the circuit shown in Fig. 1
    for p in range(reps):
        for terms, weights in terms_and_weights.items():
            if len(terms) == 1: # single-qubit terms
                
                # your code here!
                
        for terms, weights in terms_and_weights.items():
            if len(terms) == 2: # two-qubit terms
                
                # your code here!
                
        qc.rx(2 * betas[p], range(num_qubits)) # mixer 
    qc = qc.reverse_bits() #Because the measurement is given in reversed order
    return qc

def sympy_to_dict(ising_hamiltonian):
    ising_hamiltonian = ising_hamiltonian.expand().simplify()
    n_vars = len(ising_Hamiltonian.free_symbols)
    variables = [Symbol(f"z{i}") for i in range(n_vars)]
    hamiltonian_dict = ising_Hamiltonian.as_coefficients_dict()
    max_weight = float(np.max(np.abs(list(ising_Hamiltonian.as_coefficients_dict().values()))))
    terms_and_weights = {}
    for i in range(n_vars):
        if variables[i] in hamiltonian_dict.keys():
            terms_and_weights[(i,)] = hamiltonian_dict[variables[i]]/max_weight
        for j in range(i, n_vars):
            if variables[j] * variables[i] in hamiltonian_dict.keys():
                terms_and_weights[(i,j)] = hamiltonian_dict[variables[i]*variables[j]]/max_weight
    terms_and_weights[()] = ising_Hamiltonian.args[0] # Constant term
    return terms_and_weights

### <a class="anchor" id="qaoa_circ"> </a>3.1 QAOA quantum Circuit

In [ ]:
p = 1
qc = qaoa_circuit(ising_Hamiltonian, reps=p)
qc.draw("mpl")

### <a class="anchor" id="qaoa_opt"> </a> 3.2 Optimization 

Once we define the QAOA circuit of the combinatorial optimization problem, the next step is to find values of $\beta$ and $\gamma$ that minimize the expectation value of the Ising Hamiltonian. Here, we use Qiskit and simpy to find the minimum of cost function. In this case, we use the COBYLA optimization method with a maximum iteration equal to 200. Then, We use method Nelder-Mead to refine the solution.

### <font color=#C4232E> Exercise 4 </font>

Evaluate the `QUBO` with the solution of the `QAOA` circuit.

In [ ]:
callback_f = {"fx":[], "params":[]}
def cost_func(parameters, circuit, objective, shots=10, backend=Aer.get_backend("qasm_simulator")):
    """
    Return a cost function that depends of the QAOA circuit 

    Parameters
    ----------
    parameters : list
        gamma and beta values of the QAOA circuit.
    circuit : QuantumCircuit
        Qiskit quantum circuit of the QAOA.
    objective : sympy QUBO formulation 
        Objective function of the problem 
    shots : int, optional
        number of times the QAOA circuit is run. The default is 10.
    backend : Qiskit Backend, optional
        The default is Aer.get_backend("qasm_simulator").

    Returns
    -------
    float
        Cost of the evaluation of n string on the objective function 

    """
    cost = 0
    # running the QAOA circuit using qiskit 
    qc = circuit.copy()
    qc.measure_all()
    counts = backend.run(qc.assign_parameters(parameters=parameters), shots=shots).result().get_counts()
    # The qiskit's result is a dictionary with strings {0,1}^n as keys and count number as the value
    for sample, n_counts in counts.items():
        dict_sol = {xi:int(bi) for xi, bi in zip(objective.free_symbols, sample)}
        
        # ----------    Your code here ---------
        
        feval = #evaluate the QUBO using the bitstring dict_sol
        
        # -----------------------------
        
        cost += n_counts * float(feval)
    callback_f["fx"].append(cost/shots)
    callback_f["params"].append(parameters)
    return cost / shots


In [ ]:
opt_steps = {"fx":[], "params":[]}
Nfeval = 0
def callback(parameters):
    # Saving the results of every iteration
    global Nfeval
    print('{0:4d}   {1: 3.6f}   {2: 3.6f}   {3: 3.6f}'.format(Nfeval, parameters[0], parameters[1], cost_func(parameters, qc, qubo)))
    Nfeval += 1
    opt_steps["fx"].append(cost_func(parameters, qc, qubo))
    opt_steps["params"].append(parameters)

np.random.seed(1)
x0 = [0.1, 0.1] # Initial guessing
#-------- Simpy minimization method to find optimal parameters for beta and gamma
sol = minimize(cost_func, x0 = x0, args=(qc, qubo, 100),
               callback=callback, method="COBYLA", options={"maxiter":200, "rhobeg":np.pi/2})
#refine the solution with a second optimization method
sol = minimize(cost_func, x0 = sol.x, args=(qc, qubo, 100),
               callback=callback, method="nelder-mead", options={"maxiter":100})
sol


### <a class="anchor" id="visualization"> </a> 3.3 Results visualization

#### Optimization steps (Ising Hamiltonian expectation value)

In [ ]:
# expectation value vs. iterations
fig, ax = plt.subplots()
ax.plot(opt_steps["fx"])
ax.set_xlabel("iterations")
ax.set_ylabel(r"$\langle H(z)\rangle$")
ax.grid()
ax.set_title("QAOA")

#### Histogram visualization: 50 Shots of the QAOA circuit with optimal parameters

In [ ]:
# Use the Aer simulator with 50 shots to find the distribution of solutions. Note that not all of them are valid solutions.
# Remember that QAOA in general does not find the optimal solution but a probability distribution where optimal and suboptimal
# solutions are more probable in general.
backend = Aer.get_backend("qasm_simulator")
arg_min = np.argmin(callback_f["fx"])
params = callback_f["params"][arg_min]
qc_eval = qc.copy() # Create a copy of our qiskit quantum circuit
qc_eval.measure_all() # Measure all the qubits
results = backend.run(qc_eval.assign_parameters(parameters=sol.x), shots=50).result().get_counts() #Run the circuit on qasm_simulator backend
opt_res = {sol_str:results[sol_str]} #times the optimal solution is found
# results.pop(sol_str[::-1])
# plot_histogram is a qiskit function
fig, ax = plt.subplots(figsize=(20,5))
ax.bar([int(k, 2) for k in results.keys()], results.values())
ax.bar([int(k, 2) for k in results.keys() if k in opt_res], [v for k, v in results.items() if k in opt_res], color="tab:red", label="optimal")
ax.set_xticks(range(2**len(qubo.free_symbols)))
ticks = ax.set_xticklabels([np.binary_repr(i, len(qubo.free_symbols)) for i in range(2**len(qubo.free_symbols))], rotation=90)
ax.set_ylabel("Count", fontsize=18)
ax.legend()
ax.set_title("QAOA with optimized parameters solutions count", fontsize=18)

In [ ]:
# statevector_simulator gives us the exact solution of QAOA, additionally, from it, we can get the probabilities.
backend = Aer.get_backend("statevector_simulator")
probabilities = backend.run(qc.assign_parameters(parameters=sol.x)).result().get_statevector().probabilities()
print(f"Probability of finding the optimal solution using QAOA: {100*np.round(probabilities[int(sol_str,2)],4)}%")
print(f"Random guessing: {100/2**len(sol_str)}%")

## Landscape

For the case where there is just one layer on the QAOA, we can visualize the energy expectation value $\langle H(z) \rangle$ for the knapsack problem. The Figure below shows the landscape for the Knapsack problem with the optimal solution of the optimization step.

### <font color=#C4232E> Exercise 5 </font>

Using the `cost_func` evaluate the energy landscape for different gammas and betas given below.

In [ ]:
n1 = n2 = 50
gammas = np.linspace(0, np.pi, n1)
betas = np.linspace(-np.pi/2, np.pi/2, n2)

landscape = np.zeros((n1, n2))
circ = qc.copy()
circ.measure_all()
for i in range(n1):
    for j in range(n2):
        
        landscape[i,j] = # your code here!

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax1 = ax.imshow(landscape, cmap="coolwarm", extent=[-np.pi/2, np.pi/2, np.pi, 0])
ax.plot(sol.x[1], sol.x[0], marker="*", markersize=10, markeredgecolor="black", color="tab:red", label="optimal", linewidth=0)

ax.set_xticks([-np.pi/2, 0, np.pi/2])
ax.set_yticks([0,np.pi])
ax.set_xticklabels([r"$-\pi/2$", 0, r"$\pi/2$"])
ax.set_yticklabels([0, r"$\pi$"])
ax.set_xlabel(r"$\beta$")
ax.set_ylabel(r"$\gamma$")
ax.legend(loc='upper center', bbox_to_anchor=(0.5,1.27))
ax.set_title("Energy Landscape", fontsize=18)
plt.colorbar(ax1)

## <a class="anchor" id="task3"></a> Exercise 6 (Optional)

In this excercise you must repeat the knapsack problem this time with some modifications. Now, we need to maximize the value transported on a knapsack with 6 items with weights $w = [7, 2, 1, 3, 2, 5]$, values $v = [4, 3, 2, 1, 5, 3]$, and maximum weight $W_{max} = 15$. An additional restriction in this case is that just one of the items $[x_1, x_3, x_5]$ could be in the knapsack.

\begin{equation}
x_1 + x_3 + x_5 = 1
\end{equation}

1. Repeat the steps in [**Example**](#example) and [**QAOA**](#qaoa) for this problem (note that you should modify the qubo formulation to include the equality constraint.)

2. Run it on a [fake backend](https://qiskit.org/documentation/apidoc/providers_fake_provider.html) to simulate what happen on a real quantum device. Please use the fakle backend of `ibmq_guadalupe`

In [ ]:
from qiskit.providers.fake_provider import FakeGuadalupeV2

In [ ]:
backend = FakeGuadalupeV2()